In [1]:
from split import split_data, is_dir_empty

SRC_MINE_DIR = './dataset/mine/images'
TRAINING_DIR = "./dataset/mine/training/"
VALIDATION_DIR = "./dataset/mine/validation/"
TEST_DIR = "./dataset/mine/test/"

# 检查训练集和验证集目录是否为空
train_dir_empty = is_dir_empty(TRAINING_DIR)
val_dir_empty = is_dir_empty(VALIDATION_DIR)

if train_dir_empty and val_dir_empty:
    print("Training and Validation directories are empty. Starting data split...\n")
    split_data(SRC_MINE_DIR, TRAINING_DIR, VALIDATION_DIR,
               test_dir=None, include_test_split=False, split_ratio=0.8)
else:
    # 提示目录不为空的原因
    if not train_dir_empty:
        print(f"❌ Training directory ({TRAINING_DIR}) is not empty!")
    if not val_dir_empty:
        print(f"❌ Validation directory ({VALIDATION_DIR}) is not empty!")
    print("\nSkip data split. Please empty the directories first if you want to re-run the split.")
    

❌ Training directory (./dataset/mine/training/) is not empty!
❌ Validation directory (./dataset/mine/validation/) is not empty!

Skip data split. Please empty the directories first if you want to re-run the split.


In [2]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
validation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder(TRAINING_DIR, transform=train_transform)
validation_dataset = ImageFolder(VALIDATION_DIR,transform=validation_transform)

BATCH_SIZE = 64
NUM_WORKERS = 15
PIN_MEMORY = True

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS, 
    pin_memory=PIN_MEMORY
)

validation_loader = DataLoader(
    validation_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS, 
    pin_memory=PIN_MEMORY
)

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math

print("=== 数据集详情 ===")
print(f"训练集数量: {len(train_dataset)}")
print(f"验证集数量: {len(validation_dataset)}")
print(f"分类类别: {train_dataset.classes}")
print(f"类别索引: {train_dataset.class_to_idx}")

def show_batch_with_labels(loader, dataset_classes, title="Batch"):
    # 从 DataLoader 获取一个 batch 的数据
    images, labels = next(iter(loader))
    batch_size = len(images)
    
    print(f"\n--- {title} 信息 ---")
    print(f"图片 Tensor 形状: {images.shape}")
    print(f"标签 Tensor 形状: {labels.shape}")
    
    cols = 8
    rows = math.ceil(batch_size / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2.5, rows * 3))
    fig.suptitle(title, fontsize=16)
    
    axes = axes.flatten() if batch_size > 1 else [axes]
        
    for i in range(len(axes)):
        ax = axes[i]
        if i < batch_size:
            img = images[i].numpy().transpose((1, 2, 0))
            
            ax.imshow(img)
            
            label_name = dataset_classes[labels[i]]
            
            ax.set_xlabel(label_name, fontsize=11, fontweight='bold')
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.subplots_adjust(top=0.92) 
    plt.show()

# show_batch_with_labels(train_loader, train_dataset.classes, "Training")
# show_batch_with_labels(validation_loader, validation_dataset.classes, "Validation")

=== 数据集详情 ===
训练集数量: 4510
验证集数量: 1130
分类类别: ['biotite', 'bornite', 'chrysocolla', 'malachite', 'muscovite', 'pyrite', 'quartz']
类别索引: {'biotite': 0, 'bornite': 1, 'chrysocolla': 2, 'malachite': 3, 'muscovite': 4, 'pyrite': 5, 'quartz': 6}


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import MineralCNN, fit_model

# 1. 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 准备数据信息
class_names = train_dataset.classes
num_classes = len(class_names)

# 3. 实例化模型
cnn_model = MineralCNN(num_classes=num_classes).to(device)

# 4. 训练配置
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# 5. 开始训练 (注意新增加了 class_names 参数)
history = fit_model(
    cnn_model, 
    train_loader, 
    validation_loader, 
    criterion, 
    optimizer, 
    num_epochs=50, # 可以根据需要调整
    device=device,
    class_names=class_names # 用于生成混淆矩阵标签
)

Using device: cuda
Training started. Results will be saved to: runs/train6


Epoch [1/50] | T_Loss: 1.5338 T_Acc: 43.61% | V_Loss: 1.3462 V_Acc: 49.56% | LR: 0.001000


Epoch [2/50] | T_Loss: 1.3006 T_Acc: 53.66% | V_Loss: 1.5110 V_Acc: 46.02% | LR: 0.001000


Epoch [3/50] | T_Loss: 1.2615 T_Acc: 54.94% | V_Loss: 1.3807 V_Acc: 48.58% | LR: 0.001000


Epoch [4/50] | T_Loss: 1.1837 T_Acc: 58.05% | V_Loss: 1.2699 V_Acc: 55.66% | LR: 0.001000


...

Epoch [46/50] | T_Loss: 0.0283 T_Acc: 98.80% | V_Loss: 0.5162 V_Acc: 90.00% | LR: 0.000125


Epoch [47/50] | T_Loss: 0.0260 T_Acc: 98.63% | V_Loss: 0.5074 V_Acc: 90.09% | LR: 0.000125


Epoch [48/50] | T_Loss: 0.0242 T_Acc: 98.89% | V_Loss: 0.5055 V_Acc: 90.35% | LR: 0.000125


Epoch [49/50] | T_Loss: 0.0243 T_Acc: 98.80% | V_Loss: 0.5172 V_Acc: 89.73% | LR: 0.000125


Epoch [50/50] | T_Loss: 0.0264 T_Acc: 98.78% | V_Loss: 0.5065 V_Acc: 90.62% | LR: 0.000125

 Training finished. Generatring reports...
Loading best model for evaluation...
Saving confusion matrix...
Saving prediction samples...
 All done! Check directory: runs/train6


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import importlib
import model
importlib.reload(model) # 重新加载以应用更改
from modelCBAM import MineralCBAMResNet, fit_model

# 1. 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 准备数据信息
class_names = train_dataset.classes
num_classes = len(class_names)

# 3. 实例化创新模型 (CBAM-ResNet)
# 注意这里调用的是 MineralCBAMResNet
cbam_model = MineralCBAMResNet(num_classes=num_classes).to(device)

# 4. 训练配置
criterion = nn.CrossEntropyLoss()
# 学习率可以保持 0.001
optimizer = optim.Adam(cbam_model.parameters(), lr=0.001)

# 5. 开始训练
print(">>> 开始训练 CBAM-ResNet 创新架构 <<<")
# 建议先跑 50 个 Epoch 进行对比
history_cbam = fit_model(
    cbam_model, 
    train_loader, 
    validation_loader, 
    criterion, 
    optimizer, 
    num_epochs=50, 
    device=device,
    class_names=class_names
)

Using device: cuda
>>> 开始训练 CBAM-ResNet 创新架构 <<<
Training started (CBAM-ResNet). Results will be saved to: runs/train_cbam


Epoch [1/50] | T_Loss: 1.3937 T_Acc: 47.85% | V_Loss: 1.6074 V_Acc: 46.37% | LR: 0.001000


Epoch [2/50] | T_Loss: 1.1708 T_Acc: 57.80% | V_Loss: 1.9923 V_Acc: 38.32% | LR: 0.001000


Epoch [3/50] | T_Loss: 1.0325 T_Acc: 63.15% | V_Loss: 1.4878 V_Acc: 41.77% | LR: 0.001000


Epoch [4/50] | T_Loss: 0.8609 T_Acc: 70.00% | V_Loss: 2.4926 V_Acc: 34.78% | LR: 0.001000


...

Epoch [46/50] | T_Loss: 0.0226 T_Acc: 99.00% | V_Loss: 0.4585 V_Acc: 90.62% | LR: 0.000008


Epoch [47/50] | T_Loss: 0.0243 T_Acc: 98.80% | V_Loss: 0.4527 V_Acc: 90.53% | LR: 0.000008


Epoch [48/50] | T_Loss: 0.0235 T_Acc: 99.00% | V_Loss: 0.4684 V_Acc: 90.71% | LR: 0.000004


Epoch [49/50] | T_Loss: 0.0231 T_Acc: 98.89% | V_Loss: 0.4506 V_Acc: 90.71% | LR: 0.000004


Epoch [50/50] | T_Loss: 0.0233 T_Acc: 98.89% | V_Loss: 0.4568 V_Acc: 90.53% | LR: 0.000004

 Training finished. Generatring reports...
Loading best model for evaluation...
Saving confusion matrix...
 All done! Check directory: runs/train_cbam
